In [1]:
TRAIN = True

## MODEL py

In [2]:
# Pix2Pix tutorial
#https://www.youtube.com/watch?v=YsrMGcgfETY
# Pix2Pix paper
#https://arxiv.org/pdf/1611.07004.pdf
# Paper U-Net: Convolutional Networks for BiomedicalImage Segmentation
#https://arxiv.org/pdf/1505.04597.pdf

In [2]:
import numpy as np 
import os
import skimage.io as io
#import skimage.transform as trans
from datetime import datetime
import numpy as np
import tensorflow as tf 
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard
from tensorflow.keras.utils import plot_model
#from ktf import backend as keras   .add(lambda(lambda x: x ** 2))

def downsample(n_filers, kernel=3, apply_batch=True):
    initializer = tf.random_normal_initializer(0,0.02)
    result = Sequential()
    result.add(Conv2D(n_filers,
                    kernel,
                    strides = 2,
                    padding = 'same',
                    kernel_initializer = initializer,
                    use_bias = not apply_batch,))
    if apply_batch:
        result.add(BatchNormalization())
    result.add(ReLU())
    return result

def upsample(n_filers, kernel = 3, apply_dropout=False):
    initializer = tf.random_normal_initializer(0,0.02)
    result = Sequential()
    result.add(Conv2DTranspose(n_filers,
                            kernel,
                            strides = 2,
                            padding = 'same',
                            kernel_initializer = initializer,
                            use_bias = False,))
    result.add(BatchNormalization())
    if apply_dropout:
        result.add(Dropout(0.5))
    result.add(ReLU())
    return result

def Segmenter(pretrained_weights = None,input_size = (256,256,3)):
    inputs = Input(input_size)
    down_stack=[
              #Lambda(lambda x: x/127.5-1),           #Normaliza los valores de los pixeles
              downsample(64,apply_batch=False),  #128,
              downsample(128),                   #64
              downsample(256),                   #32
              downsample(512),                   #16
              downsample(512),                   #8
              downsample(512),                   #4
              downsample(512),                   #2
              downsample(512),                   #1
    ]
    up_stack=[
              upsample(512,apply_dropout=True), #2,
              upsample(512,apply_dropout=True), #4
              upsample(512,apply_dropout=True), #8
              upsample(512),                   #16
              upsample(256),                   #32
              upsample(128),                   #64
              upsample(64),                   #128
    ]
    initializer = tf.random_normal_initializer(0,0.02)
    last = Conv2DTranspose(filters =1,
                         kernel_size = 3,
                         strides = 2,
                         padding='same',
                         kernel_initializer=initializer,
                         activation = 'sigmoid',)
    x = inputs
    s =[]
    concat = Concatenate()
    # CODIFICADOR
    for down in down_stack:
        x = down(x)
        s.append(x)
    s = s[::-1][1:]
    # DECODIFICADOR
    for up,sk in zip(up_stack,s):
        x = up(x)
        x = concat([x,sk])
    # Capa final de binarización
    last =  last(x)
    # Generando el modelo
    model = Model(inputs=inputs, outputs = last)
    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    # load pretrained weights
    if(pretrained_weights):
        model.load_weights(pretrained_weights)
    return model

## DATA py

In [3]:
from __future__ import print_function
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import os
import glob
import skimage.io as io
from os import walk
import skimage.transform as trans

Sky = [128,128,128]
Building = [128,0,0]
Pole = [192,192,128]
Road = [128,64,128]
Pavement = [60,40,222]
Tree = [128,128,0]
SignSymbol = [192,128,128]
Fence = [64,64,128]
Car = [64,0,128]
Pedestrian = [64,64,0]
Bicyclist = [0,128,192]
Unlabelled = [0,0,0]

COLOR_DICT = np.array([Sky, Building, Pole, Road, Pavement,
                          Tree, SignSymbol, Fence, Car, Pedestrian, Bicyclist, Unlabelled])


def adjustData(img,mask,flag_multi_class,num_class):
    if(flag_multi_class):
        img = img / 255
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):
            #for one pixel in the image, find the class in mask and convert it into one-hot vector
            #index = np.where(mask == i)
            #index_mask = (index[0],index[1],index[2],np.zeros(len(index[0]),dtype = np.int64) + i) if (len(mask.shape) == 4) else (index[0],index[1],np.zeros(len(index[0]),dtype = np.int64) + i)
            #new_mask[index_mask] = 1
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img) > 1):
        img = img / 255
        mask = mask /255
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    return (img,mask)



def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "rgb",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (256,256),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path" ImageDataGenerator.flow_from_directory
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)



def testGenerator(test_path,num_image = 30,target_size = (256,256),flag_multi_class = False,as_gray = True):
    f=[]
    for (dirpath, dirnames, filenames) in walk(test_path):
        f.extend(filenames)
        break
    for name in f:
        img = io.imread(os.path.join(test_path,name),as_gray = as_gray)
        img = img / 255
        img = cv2.resize(img,target_size[::-1])
        #img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        img = np.reshape(img,(1,)+img.shape)
        yield img


def geneTrainNpy(image_path,mask_path,flag_multi_class = False,num_class = 2,image_prefix = "image",mask_prefix = "mask",image_as_gray = True,mask_as_gray = True):
    image_name_arr = glob.glob(os.path.join(image_path,"%s*.png"%image_prefix))
    image_arr = []
    mask_arr = []
    for index,item in enumerate(image_name_arr):
        img = io.imread(item,as_gray = image_as_gray)
        img = np.reshape(img,img.shape + (1,)) if image_as_gray else img
        mask = io.imread(item.replace(image_path,mask_path).replace(image_prefix,mask_prefix),as_gray = mask_as_gray)
        mask = np.reshape(mask,mask.shape + (1,)) if mask_as_gray else mask
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        image_arr.append(img)
        mask_arr.append(mask)
    image_arr = np.array(image_arr)
    mask_arr = np.array(mask_arr)
    return image_arr,mask_arr


def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out / 255


def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2,orig_size=(255,255)):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        img = cv2.resize(img,orig_size[::-1])
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)
        
#Post procesamiento de la mascara obtenida        
def post_mask(img,orig_size=(255,255),flag_multi_class = False,num_class = 2, threshold = 0.5 ):
    img = labelVisualize(num_class,COLOR_DICT,img) if flag_multi_class else img[:,:,0]
    img[img<threshold] = 0
    img[img>=threshold] = 255
    img, area = post_mask_util(img)
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = cv2.resize(img,orig_size[::-1], interpolation = cv2.INTER_AREA) #resize en opencv esta invertido
    img = img.astype('uint8') 
    return img,area

def post_mask_util(img_bin, kernel_=(3,3)):
    # Clean up
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_)
    # Fill small gaps
    img_bin = cv2.morphologyEx(img_bin, cv2.MORPH_CLOSE, kernel)
    # Remove specks
    img_bin = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, kernel)
    # Contando la cantidad de puntos detectados en la mascara de 256x256
    area = np.sum(img_bin == 255)
    
    return img_bin , area
# Preprocesamiento para el tratado de la mascara
def pre_mask(img_orig, crop = (.3 , .87, .38,.56),size=(256,256)):
    (alto, ancho, chs) = img_orig.shape
    y_ini = int(alto*crop[0])
    y_fin = int(alto*crop[1])
    x_ini = int(ancho*crop[2])
    x_fin = int(ancho*crop[3])
    img_crop = img_orig[y_ini:y_fin,x_ini:x_fin]
    img = img_crop.copy()
    img = img / 255
    img = cv2.resize(img,size, interpolation = cv2.INTER_AREA)
    #img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
    img = np.reshape(img,(1,)+img.shape)
    return img , img_crop # imagen a introducir en la red , dimensiones originales de la imagen de entrada

integer = 0
last_time = 0
worst_lost = 0
#Función de integración para la integración de areas en el tiempo
def lost_condition(area, time):
    # se toma como medida de referencia una taza de 30FPS
    global last_time
    global integer
    global worst_lost
    if area > 30:
        integer += area*(time-last_time)
    else:
        integer = 0 
        if worst_lost < integer: 
            worst_lost = integer

## RUN INFERENCE

In [ ]:
import cv2
import time
model = Segmenter(pretrained_weights ="segmenter_unet.hdf5",input_size = (256,256,3))
set_ = 0
#pos_cam = [(.3 , .78, .38,.56), (.25,.5,.6,.7)]
pos_cam = [(.3 , .87, .38,.56), (.25,.5,.6,.7)]
pos    = pos_cam[set_]
videos = ["cheves.mp4", "cheves2.mp4"]
camara = cv2.VideoCapture(os.path.join("img_orig",videos[set_]))
while True and not TRAIN:
    last_time = time.time()
    (grabbed, frame) = camara.read()
    if not grabbed:
        break
    time_pre_start = time.time()
    img, img_crop = pre_mask(frame, crop = pos)
    time_pre_end = time.time()
    results = model.predict(img)
    time_predic_end = time.time()
    
    rgb_mask, area = post_mask(results[0],orig_size=img_crop.shape[:2],threshold=0.9)
    lost_condition(area, time.time())
    time_post_end = time.time()
    pre = int((time_pre_end - time_pre_start)*1000)
    pred = int((time_predic_end - time_pre_end)*1000) 
    posp = int((time_post_end - time_predic_end)*1000) 
    print("Time_pre = {} Time_predic: {} Time_posp: {} Indice fuga: {}".format(pre,pred,posp,integer))

    img = cv2.addWeighted(rgb_mask, 0.6, img_crop, 0.4, 0)
    cv2.imshow("origi + Mask",img)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("s"):
        break
camara.release()
cv2.destroyAllWindows()

Time_pre = 1 Time_predic: 390 Time_posp: 2 Indice fuga: 610.571364402771
Time_pre = 0 Time_predic: 102 Time_posp: 1 Indice fuga: 676.1361827850342
Time_pre = 0 Time_predic: 105 Time_posp: 2 Indice fuga: 0
Time_pre = 1 Time_predic: 111 Time_posp: 1 Indice fuga: 0
Time_pre = 1 Time_predic: 109 Time_posp: 2 Indice fuga: 0
Time_pre = 2 Time_predic: 109 Time_posp: 2 Indice fuga: 0
Time_pre = 1 Time_predic: 102 Time_posp: 2 Indice fuga: 0
Time_pre = 2 Time_predic: 101 Time_posp: 1 Indice fuga: 0
Time_pre = 1 Time_predic: 107 Time_posp: 1 Indice fuga: 0
Time_pre = 0 Time_predic: 109 Time_posp: 1 Indice fuga: 0
Time_pre = 0 Time_predic: 107 Time_posp: 2 Indice fuga: 0
Time_pre = 1 Time_predic: 109 Time_posp: 1 Indice fuga: 0
Time_pre = 1 Time_predic: 107 Time_posp: 1 Indice fuga: 0
Time_pre = 2 Time_predic: 113 Time_posp: 1 Indice fuga: 0
Time_pre = 2 Time_predic: 109 Time_posp: 1 Indice fuga: 0
Time_pre = 2 Time_predic: 125 Time_posp: 1 Indice fuga: 0
Time_pre = 1 Time_predic: 117 Time_posp: 

In [ ]:
a=[1,2]
a[::-1]

## Train model

In [5]:
if TRAIN:
    data_gen_args = dict(rotation_range=0.2,
                        width_shift_range=0.05,
                        height_shift_range=0.05,
                        shear_range=0.05,
                        zoom_range=0.05,
                        horizontal_flip=True,
                        fill_mode='nearest')
    myGene = trainGenerator(2,'data/train','img2','label2',data_gen_args,save_to_dir = None)
    myGene_val = trainGenerator(2,'data/train','img','label',data_gen_args,save_to_dir = None, seed=45)

    #model = Segmenter(input_size = (256,256,3))
    model = Segmenter(pretrained_weights ="segmenter_unet.hdf5",input_size = (256,256,3))
    logdir="logs" 
    tboard_callback = TensorBoard(log_dir=logdir)
    model_checkpoint = ModelCheckpoint('segmenter_unet.hdf5', monitor='val_loss',verbose=1, save_best_only=True) 
    model.fit(myGene,validation_data=myGene_val, steps_per_epoch=700,epochs=25,callbacks=[model_checkpoint,tboard_callback],validation_steps=400)      

    testGene = testGenerator("data/test",as_gray = False)
    results = model.predict_generator(testGene,30,verbose=1)
    saveResult("data/test/tested",results,orig_size=(169,115))

Found 218 images belonging to 1 classes.
Found 218 images belonging to 1 classes.


W0130 05:17:59.551648 140080063682304 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']


Found 56 images belonging to 1 classes.
Found 56 images belonging to 1 classes.


W0130 05:17:59.817475 140080063682304 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']


Train for 700 steps, validate for 400 steps
Epoch 1/25
699/700 [============================>.] - ETA: 0s - loss: 0.0090 - accuracy: 0.9963
Epoch 00001: val_loss improved from inf to 0.03447, saving model to segmenter_unet.hdf5
700/700 [==============================] - 382s 545ms/step - loss: 0.0090 - accuracy: 0.9963 - val_loss: 0.0345 - val_accuracy: 0.9888
Epoch 2/25
699/700 [============================>.] - ETA: 0s - loss: 0.0090 - accuracy: 0.9963
Epoch 00002: val_loss improved from 0.03447 to 0.02467, saving model to segmenter_unet.hdf5
700/700 [==============================] - 380s 544ms/step - loss: 0.0090 - accuracy: 0.9963 - val_loss: 0.0247 - val_accuracy: 0.9911
Epoch 3/25
699/700 [============================>.] - ETA: 0s - loss: 0.0088 - accuracy: 0.9963
Epoch 00003: val_loss did not improve from 0.02467
700/700 [==============================] - 365s 522ms/step - loss: 0.0088 - accuracy: 0.9963 - val_loss: 0.0332 - val_accuracy: 0.9883
Epoch 4/25
699/700 [============

W0130 06:52:27.508926 140080063682304 callbacks.py:1018] Can save best model only with val_loss available, skipping.


KeyboardInterrupt: 

In [ ]:
#!tensorboard --logdir logs/train

In [ ]:
model = Segmenter(pretrained_weights ="segmenter_unet.hdf5",input_size = (256,256,3))
testGene = testGenerator("data/test",as_gray = False)
results = model.predict_generator(testGene,28,verbose=1)
saveResult("data/test/tested",results,orig_size=(169,115))